# COSMOS master catalogue for Farmer photometry
## Convert COSMOS2020 to help format for HELP DR2 alpha

This catalogue comes from `dmu1_COSMOS2020`. We here process the COSMOS2020 catalogue to the HELP format so that it can be merged in with the full area HELP cosmos data sets.



In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
1407877 (Mon Feb 4 12:56:29 2019 +0000)
This notebook was executed on: 
2021-01-20 18:48:53.902835


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux, flux_to_mag

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = other[key]
/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]
/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  self[key] = other[key]
/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  sel

In [3]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = '/Volumes/Raph500/COSMOS2020'
OUT_DIR = '/home/rshirley/COSMOS2020'
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "cosmos_ra"
DEC_COL = "cosmos_dec"

## I - Column selection

In [7]:
bands = OrderedDict({
    'UVISTA_Ks': 'vista_ks',
    'UVISTA_Y': 'vista_y',
    'UVISTA_H': 'vista_h',
    'UVISTA_J': 'vista_j',

    #CFHT Megacam
    'CFHT_u': 'megacam_u',
    #SUBARU Suprime
    #'SC_B': 'suprime_b',
    #'SC_V': 'suprime_v',
    #'SC_ip': 'suprime_ip',
    #'SC_rp': 'suprime_rc',
    #'SC_zp': 'suprime_zp',
    #'SC_zpp': 'suprime_zpp',
    'SC_IA484': 'suprime_ia484',
    'SC_IA527': 'suprime_ia527',
    'SC_IA624': 'suprime_ia624',
    'SC_IA679': 'suprime_ia679',
    'SC_IA738': 'suprime_ia738',
    'SC_IA767': 'suprime_ia767',
    'SC_IB427': 'suprime_ib427',
    'SC_IB464': 'suprime_ib464',
    'SC_IB505': 'suprime_ib505',
    'SC_IB574': 'suprime_ib574',
    'SC_IB709': 'suprime_ib709',
    'SC_IB827': 'suprime_ib827',
    'SC_NB711': 'suprime_nb711',
    'SC_NB816': 'suprime_nb816',
    #CFHT WIRCAM
    #'hw': 'wircam_h',
    #'ksw': 'wircam_ks',
    #SUBARU HSC
    'HSC_g': 'suprime_g',
    'HSC_r': 'suprime_r',
    'HSC_i': 'suprime_i',
    'HSC_z': 'suprime_z',
    'HSC_y': 'suprime_y',
    #Spitzer IRAC
    'IRAC_1': 'irac_i1', #'irac_i1'???
    'IRAC_2': 'irac_i2', # #irac_i2
    'SPLASH_3': 'irac_i3', # #irac_i1
    'SPLASH_4': 'irac_i4', # #irac_i4
})

In [8]:
imported_columns = OrderedDict({
    #'help_id': 'help_id', #COSMOS 2020 hasn't had help ids added
    'ID': 'cosmos_id',
    'ALPHA_J2000': 'cosmos_ra',
    'DELTA_J2000': 'cosmos_dec',
    #'class': 'cosmos_stellarity', #No stellarity in COSMOS 2020?
})

for band in bands:
  
    imported_columns.update({band + '_MAG': 'm_cosmos-' + bands[band]})
    imported_columns.update({band + '_MAGERR': 'merr_cosmos-' + bands[band]})
    imported_columns.update({band + '_FLUX': 'f_cosmos-' + bands[band]})
    imported_columns.update({band + '_FLUXERR': 'ferr_cosmos-' + bands[band]})
    imported_columns.update({band + '_CHISQ': 'chisq_cosmos-' + bands[band]})
      
        


In [9]:

catalogue = Table.read(
    OUT_DIR+"/cosmos2020.calet.org/PHOTOMETRY"
    +"/COSMOS2020_FARMER_v1.8_formatted.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2015 #Various epochs

# Clean table metadata
catalogue.meta = None

KeyError: 'SPLASH_3_CHISQ'

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    #print(col)

    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        chisqcol = "chisq{}".format(col[1:])
        
        mask = (catalogue[col] > 90.) | (catalogue[col] < 0.)
        catalogue[col][mask] = np.nan
        catalogue[errcol][mask]  = np.nan     
        
  
        
        # Band-flag column
        catalogue.add_column(Column(catalogue[chisqcol]<5, name="flag{}".format(col[1:]), dtype=bool))
        catalogue.remove_column(chisqcol)
    if col.startswith('f_'):
        errcol = "ferr{}".format(col[1:])
        
        mask = (np.isclose(catalogue[col] , -99.) )
        catalogue[col][mask] = np.nan
        catalogue[errcol][mask] = np.nan 
        
  


In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = []
for f in bands:
    if 'irac' not in bands[f]:
        SORT_COLS += ['merr_ap_cosmos-' + bands[f]]


In [ ]:
#SORT_COLS = ['merr_ap_gpc1_r', 'merr_ap_gpc1_g', 'merr_ap_gpc1_i', 'merr_ap_gpc1_z', 'merr_ap_gpc1_y']
FLAG_NAME = 'cosmos_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL,  sort_col=SORT_COLS, flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_COSMOS.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
delta_ra.to(u.deg).value

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg).value
catalogue[DEC_COL] += delta_dec.to(u.deg).value

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "cosmos_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

# VI - Saving to disk

In [ ]:
catalogue[:5]

In [ ]:
for c in catalogue.colnames:
    if 'irac' in c:
        print(c)

In [ ]:
catalogue.write("{}/COSMOS2020-Farmer_HELP.fits".format(OUT_DIR), overwrite=True)